### Trích xuất thông tin từ XML

In [1]:
# Install the ultralytics package from PyPI
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 24.1 MB/s eta 0:00:0000:01


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icdar2003/SceneTrialTrain/segmentation.xml
/kaggle/input/icdar2003/SceneTrialTrain/words.xml
/kaggle/input/icdar2003/SceneTrialTrain/locations.xml
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2523.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2480.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2553.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2619.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2526.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2607.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2679.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2670.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2660.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2533.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/IMG_2491.JPG
/kaggle/input/icdar2003/SceneTrialTrain/lfsosa_12.08.2002/

In [3]:
import xml.etree.ElementTree as ET
import os

In [4]:
def extract_data_from_xml(rootdir, xml_file):
    tree = ET.parse(os.path.join(rootdir, xml_file))
    root = tree.getroot()
    data = []
    
    img_paths = []
    img_sizes = []
    img_labels = []
    bboxs = []

    for img in root:
        bbs_of_img = []
        labels_of_img = []

        for bbs in img.findall('taggedRectangles'):
            for bb in bbs.findall('taggedRectangle'):
                x = float(bb.attrib['x'])
                y = float(bb.attrib['y'])
                w = float(bb.attrib['width'])
                h = float(bb.attrib['height'])
                bbs_of_img.append([x, y, w, h])
                labels_of_img.append(bb[0].text.lower())

        img_paths.append(os.path.join(rootdir, img[0].text))
        img_labels.append(labels_of_img)
        img_sizes.append([float(img[1].attrib['x']),
                         float(img[1].attrib['y'])])
        bboxs.append(bbs_of_img)

    return img_paths, img_sizes, img_labels, bboxs
img_paths, img_sizes, img_labels, bboxs = extract_data_from_xml('/kaggle/input/icdar2003/SceneTrialTrain','words.xml')
print("img__paths",img_paths[0])
print("img_sizes",img_sizes[0])
print("img_labels",img_labels[0])
print("bboxs",bboxs[0])

img__paths /kaggle/input/icdar2003/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG
img_sizes [1600.0, 1200.0]
img_labels ['self', 'adhesive', 'address', 'labels', '36', '89m', 'cls', '250', 'on', 'a', 'roll']
bboxs [[174.0, 392.0, 274.0, 195.0], [512.0, 391.0, 679.0, 183.0], [184.0, 612.0, 622.0, 174.0], [863.0, 599.0, 446.0, 187.0], [72.0, 6.0, 95.0, 87.0], [247.0, 2.0, 197.0, 88.0], [792.0, 0.0, 115.0, 81.0], [200.0, 848.0, 228.0, 139.0], [473.0, 878.0, 165.0, 109.0], [684.0, 878.0, 71.0, 106.0], [806.0, 844.0, 218.0, 141.0]]


### Chuyển đổi sang định dạng YOLOv11

In [5]:
def convert_to_yolo_format(img_paths, img_sizes, bboxs):
    yolo_data = []
    for i in range(len(bboxs)):
        data_img = []
        w_img, h_img = img_sizes[i]
        for bbox in bboxs[i]:
            x = bbox[0]
            y = bbox[1]
            w = bbox[2]
            h = bbox[3]

            # Convert to x_center, y_center, w, h
            x_center = x + w / 2
            y_center = y + h / 2

            # Normalize
            x_center /= w_img
            y_center /= h_img
            w /= w_img
            h /= h_img

            # Because we only have one class, we set class_id to 0
            label = 0

            # Format YOLO label
            yolo_label = f"{label} {x_center} {y_center} {w} {h}"
            data_img.append(yolo_label)
        yolo_data.append((img_paths[i], data_img))
    return yolo_data


# Define class labels
class_labels = ["text"]

yolo_data = convert_to_yolo_format(img_paths, img_sizes, bboxs)
yolo_data[0]

('/kaggle/input/icdar2003/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 ['0 0.194375 0.40791666666666665 0.17125 0.1625',
  '0 0.5321875 0.40208333333333335 0.424375 0.1525',
  '0 0.309375 0.5825 0.38875 0.145',
  '0 0.67875 0.5770833333333333 0.27875 0.15583333333333332',
  '0 0.0746875 0.04125 0.059375 0.0725',
  '0 0.2159375 0.03833333333333333 0.123125 0.07333333333333333',
  '0 0.5309375 0.03375 0.071875 0.0675',
  '0 0.19625 0.7645833333333333 0.1425 0.11583333333333333',
  '0 0.3471875 0.7770833333333333 0.103125 0.09083333333333334',
  '0 0.4496875 0.7758333333333334 0.044375 0.08833333333333333',
  '0 0.571875 0.7620833333333333 0.13625 0.1175'])

### Lưu vào folder mới

In [6]:
from  sklearn.model_selection import train_test_split
import shutil
def save_yolo_data_dir(data, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    os.makedirs(os.path.join(save_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "labels"), exist_ok=True)

    for image_path, yolo_labels in data:

        # Sao chép tệp ảnh từ thư mục gốc sang thư mục yolo
        shutil.copy(image_path, os.path.join(save_dir, "images"))

        # lấy tên file ảnh và tên file label
        image_name = os.path.basename(image_path)
        label_name = os.path.splitext(image_name)[0]

        with open(os.path.join(save_dir, "labels", f"{label_name}.txt"), "w") as f:
            for yolo_label in yolo_labels:
                f.write(yolo_label + "\n")

seed = 0
val_size = 0.2
test_size = 0.125
is_shuffle = True

train_data, val_data = train_test_split(
    yolo_data, test_size=val_size, random_state=seed, shuffle=is_shuffle)

train_data, test_data = train_test_split(
    train_data, test_size=test_size, random_state=seed, shuffle=is_shuffle)

save_dir = '/kaggle/working/datasets/yolo_data'
os.makedirs(save_dir, exist_ok=True)
save_train_dir = os.path.join(save_dir, 'train')
save_val_dir = os.path.join(save_dir, 'val')
save_test_dir = os.path.join(save_dir, 'test')

save_yolo_data_dir(train_data, save_train_dir)
save_yolo_data_dir(val_data, save_val_dir)
save_yolo_data_dir(test_data, save_test_dir)

#### Test yolo images and labels

In [7]:
# import cv2
# import os

# # Đường dẫn tới ảnh và file label
# image_path = "datasets/yolo_data/train/images/036._.jpg"  # Đường dẫn tới ảnh cụ thể
# # Đường dẫn tới file label tương ứng
# label_path = "datasets/yolo_data/train/labels/036._.txt"

# # Hàm đọc và hiển thị bounding box trên ảnh
# def visualize_yolo_label(image_path, label_path):
#     # Đọc ảnh
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Không thể đọc ảnh: {image_path}")
#         return

#     # Lấy kích thước ảnh
#     height, width, _ = image.shape

#     # Đọc file label
#     with open(label_path, "r") as f:
#         labels = f.readlines()

#     # Vẽ bounding box đầu tiên lên ảnh (1 bounding box)
#     if labels:
#         label = labels[0]  # Chỉ đọc bounding box đầu tiên
#         parts = label.strip().split()
#         class_id = int(parts[0])  # ID lớp
#         center_x, center_y, box_width, box_height = map(float, parts[1:])

#         # Chuyển đổi từ tọa độ chuẩn hóa sang tọa độ pixel
#         x1 = int((center_x - box_width / 2) * width)
#         y1 = int((center_y - box_height / 2) * height)
#         x2 = int((center_x + box_width / 2) * width)
#         y2 = int((center_y + box_height / 2) * height)

#         # Vẽ hình chữ nhật lên ảnh
#         cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         cv2.putText(image, f"Class {class_id}", (x1, y1 - 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#     else:
#         print(f"File label không chứa bounding box: {label_path}")

#     cv2.imshow("YOLO Visualization", image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


# if os.path.exists(image_path) and os.path.exists(label_path):
#     visualize_yolo_label(image_path, label_path)
# else:
#     print(f"Không tìm thấy ảnh hoặc file label!")

### Tạo file yaml
- names: tên của các class.
- nc: số lượng các class.
- path: đường dẫn đến thư mục gốc.
- train, test, val: đường dẫn đến các thư mục tương ứng.

In [8]:
import yaml
# Create data.yaml
data_yaml = {
    "path": "/kaggle/working/datasets/yolo_data",
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 1,
    "names": class_labels
}

yolo_yaml_path = os.path.join(save_dir, "data.yaml")
with open(yolo_yaml_path, "w") as f:
    yaml.dump(data_yaml, f)

In [9]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11m.pt")

# Train model
results = model.train(
    data=yolo_yaml_path,
    epochs=100,
    imgsz=640,
    cache=True,
    patience=20,
    plots=True,
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 38.8M/38.8M [00:00<00:00, 339MB/s]


Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/kaggle/working/datasets/yolo_data/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 755k/755k [00:00<00:00, 29.3MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...

    100/100      8.77G     0.6228     0.4058     0.8908         69        640: 100%|██████████| 11/11 [00:06<00:00,  1.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.10it/s]

                   all         50        279      0.905       0.86      0.918      0.705



100 epochs completed in 0.225 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.59it/s]


                   all         50        279      0.904       0.86      0.917      0.703
Speed: 0.2ms preprocess, 9.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train
